# Multi Task Model

## Imports

In [15]:
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNet

## Model parameters

In [16]:
# Log parameters
model_name = 'multiTaskModel01'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 10

## Model callbacks

In [17]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=2,
                    verbose=1)
]

## Data parameters

In [18]:
image_height = 224
image_width = 224

## Data Augmentation

In [19]:
# More data augmentation?
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomContrast(0.2),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [20]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

training_ds = keras.utils.image_dataset_from_directory(
    "../../data/m3/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 43070 files belonging to 2 classes.
Using 34456 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [21]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../../data/m3/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 43070 files belonging to 2 classes.
Using 8614 files for validation.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the test dataset

In [22]:
test_ds = keras.utils.image_dataset_from_directory(
    "../../data/m3/test/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 598 files belonging to 2 classes.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Load the ResNet model

In [23]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
mobilenet = MobileNet(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                input_tensor=None,
                pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
mobilenet.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = mobilenet(inputs, training=False)

## Creating Task 1 (Face Detection) Top Model

In [24]:
# Add Dense layer
face_head = tf.keras.layers.Dense(512, activation='relu')(base_model)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(face_head)

## Creating Task 2 (Mask Detection) Top Model

In [25]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(512, activation='relu')(base_model)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [26]:
# Add Dense layer
age_head = tf.keras.layers.Dense(1024, activation='relu')(base_model)

# Final layer for binary classification
age_outputs = keras.layers.Dense(120, activation='softmax', name='age_output')(age_head)

## Creating and compiling the final model

In [27]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 0.33

model.compile(optimizer=keras.optimizers.Adam(), # Learning Rate?
                    loss={'face_output': keras.losses.BinaryCrossentropy(), 
                          'mask_output': keras.losses.BinaryCrossentropy(),
                          'age_output': keras.losses.SparseCategoricalCrossentropy()},
                    loss_weights={'face_output': gamma, 
                                  'mask_output': gamma,
                                  'age_output': gamma}, 
                    metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenet_1.00_224 (Functional  (None, 1024)        3228864     ['input_6[0][0]']                
 )                                                                                                
                                                                                                  
 dense_3 (Dense)                (None, 512)          524800      ['mobilenet_1.00_224[1][0]']     
                                                                                            

## Training the model with the dataset

In [28]:
history = model.fit(
            training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
1077/1077 [==============================] - ETA: 0s - loss: 0.0892 - face_output_loss: 0.0871 - mask_output_loss: 0.0852 - age_output_loss: 0.0981 - face_output_accuracy: 0.9677 - mask_output_accuracy: 0.9689 - age_output_accuracy: 0.9661
Epoch 1: saving model to ../../log/cps/multiTaskModel01\latest_weights.h5
1077/1077 [==============================] - 108s 96ms/step - loss: 0.0892 - face_output_loss: 0.0871 - mask_output_loss: 0.0852 - age_output_loss: 0.0981 - face_output_accuracy: 0.9677 - mask_output_accuracy: 0.9689 - age_output_accuracy: 0.9661 - val_loss: 0.0820 - val_face_output_loss: 0.0763 - val_mask_output_loss: 0.0786 - val_age_output_loss: 0.0935 - val_face_output_accuracy: 0.9713 - val_mask_output_accuracy: 0.9696 - val_age_output_accuracy: 0.9642
Epoch 2/10
1077/1077 [==============================] - ETA: 0s - loss: 0.0601 - face_output_loss: 0.0601 - mask_output_loss: 0.0603 - age_output_loss: 0.0617 - face_output_accuracy: 0.9778 - mask_output_accuracy:

## Save the model

In [29]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../../log/saved_models/multiTaskModel01\assets


INFO:tensorflow:Assets written to: ../../log/saved_models/multiTaskModel01\assets


## Test model with test dataset

In [30]:
results = model.evaluate(test_ds)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

19/19 [==============================] - 9s 259ms/step - loss: 0.1238 - face_output_loss: 0.1199 - mask_output_loss: 0.1155 - age_output_loss: 0.1399 - face_output_accuracy: 0.9666 - mask_output_accuracy: 0.9632 - age_output_accuracy: 0.9565
Loss: 0.12384119629859924; Accuracy: 0.11991569399833679


## Predict new images

In [33]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../../data/m3/test/noFace/0_0_-1_300000.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)

1/1 [==============================] - 0s 28ms/step
